In [ ]:
!pip install pandas
!pip install numpy
!pip install seaborn
!pip install textblob

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

book_tags = pd.read_csv('book_tags.csv')
books = pd.read_csv('books.csv')
ratings = pd.read_csv('ratings.csv')
toread = pd.read_csv('toread.csv')

In [3]:
# toread
#     user_id, book_id
# books
#     book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,NonEnglish
# ratings 
#     user_id,book_id,rating
# book_tags
#     goodreads_book_id,tag_id,count


In [4]:
# i) Find book_id , goodreads_book_id for records that have null values in 'original_title' in [books.csv]

books[books['original_title'].isnull()][['book_id','goodreads_book_id']]

# ii) Remove the records in [books.csv] that have null values in 'original_title'.

books = books[~books['original_title'].isnull()]

# III) Remove the records in other files [ratings.csv],[to_read.csv],[book_tags.csv] by mapping the respective book_id/goodreads_book_id (from i)

ratings = ratings[ratings['book_id'].isin(books['book_id'])]
toread = toread[toread['book_id'].isin(books['book_id'])]
book_tags = book_tags[book_tags['goodreads_book_id'].isin(books['goodreads_book_id'])]

In [6]:
# Q7: Which book (goodreads_book_id) is marked as to-read by most users [books.csv,toread.csv] ? (N)

a = toread.groupby('book_id').count().sort_values('user_id',ascending=False).reset_index()
b = books[['book_id','goodreads_book_id']]
c = pd.merge(a,b,on='book_id')
c.head(1)['goodreads_book_id']

0    7604
Name: goodreads_book_id, dtype: int64

In [7]:
# Q12: What is the mean value of rating of all the books in the dataset based on (average_rating) [books.csv] ? (F)

books['average_rating'].mean()

3.963060200668896

In [8]:
# Q2: How many unique books are present in the dataset ? Evaluate based on the 'book_id'? [books.csv] (N)

books['book_id'].nunique()

598

In [9]:
# Q4: Which book (original_title) has the maximum number of ratings based on (work_ratings_count) [books.csv] ? (S)

books.sort_values('work_ratings_count',ascending=False).head(1)['original_title']

0    Fifty Shades of Grey
Name: original_title, dtype: object

In [10]:
# Q3: How many unique users are present in the dataset [ratings.csv] ? (N)

ratings['user_id'].nunique()

46310

In [34]:
# Q10: Which book (goodreads_book_id) has the least number of count of tags given by the user ie. the book 
# with minimum user records including all tags [book_tags.csv] ? (N)

least = book_tags.groupby('goodreads_book_id').sum().sort_values('count').reset_index()
least.head(1)['goodreads_book_id']

0    2267195
Name: goodreads_book_id, dtype: int64

In [13]:
# Q9: Which book (original_title) has the minimum (average_rating) [books.csv] ? (S)

books.sort_values('average_rating',ascending=True).head(1)['original_title']

652    Billy Budd, Sailor
Name: original_title, dtype: object

In [35]:
# Q6: Which book (original_title) has the most number of counts of tags given by the user ie. the book with 
# maximum user records including all tags [book_tags.csv,books.csv] ? (S)

most = book_tags.groupby('goodreads_book_id').sum().sort_values('count',ascending=False).reset_index()
most = pd.merge(most,books[['goodreads_book_id','original_title']],on='goodreads_book_id')
most.head(1)['original_title']

0    Moby Dick; or, The Whale
Name: original_title, dtype: object

In [33]:
# Q8: Which is the least used tag, i.e. mapped with the lowest number of books [book_tags.csv]? (In case of more than one tag, mention the tag id with the least numerical value) (N)

a = book_tags.groupby('tag_id').count().sort_values('goodreads_book_id',ascending=True).reset_index()
a.head(1)['tag_id']

0    27
Name: tag_id, dtype: int64

In [32]:
# Q14: Predict sentiment using Textblob. How many positive titles (title) are there [books.csv] ? (cut-off >0) N

from textblob import TextBlob
books['sentiment'] = books['title'].apply(lambda x: TextBlob(x).sentiment.polarity)
books[books['sentiment']>=0]['title'].count()

526